In [159]:
import json
import os
import random
import fnmatch
import openpyxl
from datetime import datetime

from urllib.request import urlopen
import urllib

import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import re

In [160]:
sns.set_style("darkgrid")

In [161]:
def import_json_to_dict(url) :
    response = urllib.request.urlopen(url)
    my_dict = json.loads(response.read())
    return my_dict

In [162]:
def clean_str(s):
    d = {
        "’": "'",
        "\xa0": " ",
        "/":","
        }
    for x in d:
        s = s.replace(x, d[x]).strip()
    return s

In [163]:
month_dict = {
    "January" : "Janvier",
"February": "Février",
"March": "Mars",
"April" : "Avril",
"May" : "Mai",
"June" : "Juin",
"July" : "Juillet",
"August" : "Août",
"September":"Septembre",
"October":"Octobre",
"November":"Novembre",
"December":"Décembre"}

In [164]:
taxo_dep_df = pd.read_csv('refs/taxo_deps.csv', dtype={'dep':str, 'reg':str})
taxo_dep_df['dep'] = taxo_dep_df['dep'].apply(lambda x: x.zfill(2))
taxo_dep_df['reg'] = taxo_dep_df['reg'].apply(lambda x: x.zfill(2))
dep_list = list(taxo_dep_df['dep'].unique())


In [165]:
taxo_reg_df = pd.read_csv('refs/taxo_regions.csv', dtype={'reg':str})
taxo_reg_df['reg'] = taxo_reg_df['reg'].apply(lambda x: x.zfill(2))
reg_list = list(taxo_reg_df['reg'].unique())


In [166]:
data_dir_path = './data/'

In [167]:
propilot_path = os.path.join("data")

In [168]:
df_dict = {}
file_list = [
"dim_activity",
"dim_tree_nodes",
"dim_top_levels",
"dim_maturities",
"dim_period",
"dim_snapshots",
"dim_effects",
"dim_properties",
"dim_states",
"dim_structures",
"fact_financials",
"fact_property_values"
]

In [169]:
for file in file_list:
    for file_csv in os.listdir(propilot_path):
        if fnmatch.fnmatch(file_csv, file + "*.csv"):
            print("File loaded : ", file_csv)
            df_dict[file] = pd.read_csv(os.path.join(propilot_path, file_csv), sep=";")

File loaded :  dim_activity.csv
File loaded :  dim_tree_nodes.csv
File loaded :  dim_top_levels.csv
File loaded :  dim_maturities.csv
File loaded :  dim_period_pp_ch.csv
File loaded :  dim_snapshots.csv
File loaded :  dim_effects.csv
File loaded :  dim_properties.csv
File loaded :  dim_states_pp_ch.csv
File loaded :  dim_structures.csv
File loaded :  fact_financials_enr_pp_ch.csv


/home/mquentin-adc/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning:

Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.



File loaded :  fact_property_values.csv


In [170]:
def format_date(raw_date):
    # Convertie un format (20210131, 2020) -> (2021-01-31, 2020)
    str_date = str(raw_date)
    if str_date == 'nan':
        return raw_date
    
    if type(str_date) == int:
        print(str_date)
    if len(str_date) == 8:
        return str_date[:4] + '-' + str_date[4:6] + '-' + str_date[6:]
    else:
        return str_date
    

df_dict['fact_financials']['period_id'] = df_dict['fact_financials']['period_id'].apply(lambda x: format_date(x))
df_dict['dim_period']['period_id'] = df_dict['dim_period']['period_id'].apply(lambda x: format_date(x))

In [171]:
# Retirer les lignes ayant un period quarter year commençant par 11 -> nonsense
df_dict['dim_period'] = df_dict['dim_period'][~df_dict['dim_period']['period_quarter_year'].str.startswith('11', na=False)]

In [172]:
df = (df_dict["fact_financials"]
      .merge(df_dict["dim_tree_nodes"], left_on="tree_node_id", right_on="tree_node_id") 
      .merge(df_dict["dim_effects"], left_on="effect_id", right_on="effect_id") 
      .merge(df_dict["dim_states"], left_on="state_id", right_on="state_id")  
      .merge(df_dict["dim_period"], left_on="period_id", right_on="period_id", how = 'left')
      .merge(df_dict["dim_structures"], left_on="structure_id", right_on="structure_id"))

In [173]:
def check_data_merge(facts, df):
    num_pp_recs = facts[facts['financials_source'] == 'proPilot'].shape[0]
    num_df_recs = df.shape[0]
    assert num_pp_recs == num_df_recs, f"Le nombre d'enregistrements proPilot diffèrent : avant {num_pp_recs} - après {num_df_recs}"
    

check_data_merge(df_dict["fact_financials"], df)

In [174]:
def extract_dep_code(expr):
    nums = re.findall(r'D\d+', expr)
    if expr.endswith('D2A'):
        return '2A'
    elif expr.endswith('D2B'):
        return '2B'
    elif expr.endswith('E00'):
        return '00'
    return nums[0][1:].zfill(2) if len(nums) > 0 else None


df['dep_code'] = df['tree_node_code'].apply(lambda x: extract_dep_code(x))

In [175]:
def check_cols(cols, df):
    assert set(cols).issubset(df.columns), f"Colonnes manquantes : {set(cols) - set(df.columns)}"

In [176]:
cols = ["tree_node_name", "structure_name", "effect_id", "state_id", "period_date", "period_month_tri", "period_month_year", "financials_cumulated_amount", "dep_code"]
check_cols(cols, df)

df=df[cols]
df.rename(columns={"period_month_year":"Date", "financials_cumulated_amount":"valeur"}, inplace=True)

In [177]:
forbidden_period_value = ["Y", "Total"]

In [178]:
def clean_mesure_name(tree_node_name):
    raw_mesure = tree_node_name.split('/')[1].strip() if '/' in tree_node_name else tree_node_name
    # nettoyage de la colonne mesure, on enlève un point surnuméraire.
    mesure = re.sub('\.', "", raw_mesure)
    mesure = clean_str(mesure)
    return mesure

In [179]:
def check_single_sum_dep_equals_nat(df_copy, mesure, indic):
    nat_values = df_copy[(df_copy['structure_name'] == 'Mesure') & 
                    (df_copy['mesure'] == mesure) &
                    (df_copy['effect_id'] == indic)][['period_date', 'valeur']].set_index('period_date').to_dict()['valeur']

    dep_indic_values = df_copy[(df_copy['structure_name'] == 'Département') & 
                            (df_copy['mesure'] == mesure) & 
                            (df_copy['effect_id'] == indic)].groupby('period_date').sum().to_dict()['valeur']

    for date in dep_indic_values.keys():
        if date not in nat_values:
            #print(f"-- National pas de {date}")
            continue
        dep_val = round(dep_indic_values[date], 2)
        nat_val = round(nat_values[date], 2)
        #assert dep_val == nat_val, f"Somme départementale : {dep_val} - Valeur récupérée nationale : {nat_val}\n{date} - {mesure} - {indic}"
        # TODO : subtituer le if-else par un assert quand on aura la confirmation que les données sont cohérentes
        if dep_val == nat_val:
            pass
            #print(f'ok - {mesure} - {indic} - {date}')
        else:
            print(f'KO - {mesure} - {indic} - {date} | {dep_val} - {nat_val}')
            

def check_sum_dep_equals_nat(df):
    """ Ce test a été rajouté après avoir constaté des différences entre somme des départements et valeurs nationales."""
    df_copy = df.copy()
    df_copy = df_copy.loc[
                (~df_copy.period_month_tri.isin(forbidden_period_value)) &
                (df_copy.state_id == 'Valeur Actuelle') &
                (~df_copy.valeur.isna())].copy()

    df_copy["mesure"] = df_copy["tree_node_name"].apply(lambda x: clean_mesure_name(x))

    mesures = df_copy['mesure'].unique()
    for mesure in mesures:
        indics = df_copy[df_copy['mesure'] == mesure]['effect_id'].unique()
        for indic in indics:
            check_single_sum_dep_equals_nat(df_copy, mesure, indic)

            
check_sum_dep_equals_nat(df)

KO - Réhabilitation Friches (urbaines et sites pollués) - Nombre de sites concernés - FFR1 - 2021-07-31T00:00:00.0000000 | 111.0 - 120.0
KO - Hydrogène – Ecosystèmes territoriaux - Montant total de CO2 évité - ETH1 - 2020-12-31T00:00:00.0000000 | 121523.0 - 121520.0
KO - Hydrogène – Ecosystèmes territoriaux - Montant total de CO2 évité - ETH1 - 2021-03-31T00:00:00.0000000 | 762271.0 - 801520.0
KO - Hydrogène – Ecosystèmes territoriaux - Nombre de projets soutenus - ETH2 - 2021-03-31T00:00:00.0000000 | 20.0 - 21.0
KO - Rénovation énergétique et réhabilitation lourde des logements sociaux - Nombre de logements subventionnés - RER1 - 2021-05-31T00:00:00.0000000 | 5429.0 - 6371.0
KO - Rénovation énergétique et réhabilitation lourde des logements sociaux - Nombre de logements subventionnés - RER1 - 2021-06-30T00:00:00.0000000 | 7024.0 - 11048.0
KO - Rénovation énergétique et réhabilitation lourde des logements sociaux - Nombre de logements subventionnés - RER1 - 2021-07-31T00:00:00.0000000 

In [180]:
df.rename(columns={"effect_id":"indicateur"}, inplace=True)
df.indicateur = df.indicateur.str.strip()
df["short_indic"] = df.indicateur.apply(lambda x: x.split("-")[0].strip())
df["indic_id"] = df.indicateur.apply(lambda x: x.split("-")[1].strip())

In [181]:
df = df.loc[(~df.period_month_tri.isin(forbidden_period_value)) &
    (df.state_id == 'Valeur Actuelle') &
    (~df.valeur.isna())].copy()

In [182]:
# Filtrer les lignes ayant une date ultérieure à la date d'aujourd'hui
today = datetime.today().strftime('%Y-%m-%d')

date_series = df['period_date'].apply(lambda x: x.split('T')[0])
df['format_date'] = pd.to_datetime(date_series)

df = df[df['format_date'] <= today]
df = df.drop('format_date', axis=1)

df["departement"] = df["tree_node_name"].apply(lambda x: x.split("/")[0].strip())
df["mesure"] = df["tree_node_name"].apply(lambda x: x.split("/")[-1].strip())
df.drop(columns=["tree_node_name"], inplace=True)

# nettoyage de la colonne mesure, on enlève un point surnuméraire.
df["mesure"].replace("\.", "", regex=True, inplace=True)
df.mesure = df.mesure.apply(lambda x: clean_str(x))

# traduit les mois dans la colonne date
df.Date = df.Date.replace(month_dict, regex=True)

# enrichit avec les noms de département/région
df = df.merge(taxo_dep_df[["dep", "reg", "libelle"]], 
                          how="left", left_on="dep_code", right_on="dep")\
                   .merge(taxo_reg_df[["reg", "nccenr"]], how="left", left_on="reg", right_on="reg")

df.rename(columns={"nccenr":"region"}, inplace=True)
df.drop(['dep_code'], axis=1, inplace=True)  # Supprime code dep utilisé pour la jointure

In [183]:
df_dep = df.loc[df.structure_name == "Département"].copy()
df_dep.drop(columns=["structure_name"], inplace=True)

dep_indic = set(df_dep.indicateur.unique())
df_reg = df.loc[(df.structure_name == "Région") 
       & (~df.indicateur.isin(dep_indic))].copy()
df_reg.drop(columns=["structure_name"], inplace=True)

reg_indic = set(df_reg.indicateur.unique())
df_nat = df.loc[(df.structure_name == "Mesure") 
       & (~df.indicateur.isin(reg_indic))
       & (~df.indicateur.isin(dep_indic))].copy()
df_nat.drop(columns=["structure_name"], inplace=True)

In [184]:
df_dep["short_indic"] = df_dep.indicateur.apply(lambda x: x.split("-")[0].strip())
df_dep.short_indic = df_dep.short_indic.apply(lambda x: clean_str(x))

df_reg["short_indic"] = df_reg.indicateur.apply(lambda x: x.split("-")[0].strip())
df_reg.short_indic = df_reg.short_indic.apply(lambda x: clean_str(x))

df_nat2 = df_nat.copy() # Opérations faites sur df_nat. df_nat utile pour autres choses. 
df_nat2["short_indic"] = df_nat2.indicateur.apply(lambda x: x.split("-")[0].strip())
df_nat2.short_indic = df_nat2.short_indic.apply(lambda x: clean_str(x))

In [185]:
def check_single_sum_dep_equals_nat_2(df_copy, df_nat, mesure, indic):
    nat_values = df_nat[(df_nat['mesure'] == mesure) &
                    (df_nat["indicateur"] == indic)][['period_date', 'valeur']].set_index('period_date').to_dict()['valeur']

    dep_indic_values = df_copy[(df_copy['mesure'] == mesure) & 
                                (df_copy['indicateur'] == indic)].groupby('period_date').sum().to_dict()['valeur']

    for date in dep_indic_values.keys():
        if date not in nat_values:
            #print(f"-- National pas de {date}")
            continue
        dep_val = round(dep_indic_values[date], 2)
        nat_val = round(nat_values[date], 2)
        #assert dep_val == nat_val, f"Somme départementale : {dep_val} - Valeur récupérée nationale : {nat_val}\n{date} - {mesure} - {indic}"
        # TODO : subtituer le if-else par un assert quand on aura la confirmation que les données sont cohérentes
        if dep_val == nat_val:
            pass
            #print(f'ok - {mesure} - {indic} - {date}')
        else:
            print(f'KO - {mesure} - {indic} - {date} | {dep_val} - {nat_val}')


def check_sum_dep_equals_nat_2(df_dep, df_nat):
    mesures = df_dep['mesure'].unique()
    for mesure in mesures:
        indics = df_dep[df_dep['mesure'] == mesure]['indicateur'].unique()
        for indic in indics:
            check_single_sum_dep_equals_nat_2(df_dep, df_nat, mesure, indic)
            
            
check_sum_dep_equals_nat_2(df_dep, df)

KO - Réhabilitation Friches (urbaines et sites pollués) - Nombre de sites concernés - FFR1 - 2021-07-31T00:00:00.0000000 | 111.0 - 120.0
KO - Hydrogène – Ecosystèmes territoriaux - Montant total de CO2 évité - ETH1 - 2020-12-31T00:00:00.0000000 | 121523.0 - 121520.0
KO - Hydrogène – Ecosystèmes territoriaux - Montant total de CO2 évité - ETH1 - 2021-01-31T00:00:00.0000000 | 121523.0 - 0.0
KO - Hydrogène – Ecosystèmes territoriaux - Montant total de CO2 évité - ETH1 - 2021-02-28T00:00:00.0000000 | 121523.0 - 0.0
KO - Hydrogène – Ecosystèmes territoriaux - Montant total de CO2 évité - ETH1 - 2021-03-31T00:00:00.0000000 | 762271.0 - 801520.0
KO - Hydrogène – Ecosystèmes territoriaux - Montant total de CO2 évité - ETH1 - 2021-04-30T00:00:00.0000000 | 762271.0 - 20000.0
KO - Hydrogène – Ecosystèmes territoriaux - Montant total de CO2 évité - ETH1 - 2021-05-31T00:00:00.0000000 | 762271.0 - 20000.0
KO - Hydrogène – Ecosystèmes territoriaux - Montant total de CO2 évité - ETH1 - 2021-06-30T00:0

In [186]:
dict_indicateur = {'Nombre de repas servis dans les restaurants universitaires au tarif à 1€' : "Nombre de repas servis",
 'Montant cumulé de l’investissement total ainsi déclenché' : "Montant cumumé de l'investissement total",
 'Nombre d’entreprises bénéficiaires du dispositif' : "Nombre d'entreprises bénéficiaires",
 'Nombre de bâtiments Etat dont le marché de rénovation est notifié' : 'Nombre de bâtiments dont le marché de rénovation est notifié',
 'Nombre de projets incluant une transformation de la ligne de production pour réduire son impact environnemental' : 'Nombre de projets',
 'Nombre d\'exploitations certifiées "haute valeur environnementale"' : 'Nombre d’exploitations certifiées',
 'Emissions de gaz à effet de serre évitées sur la durée de vie des équipements' : 'Emissions de gaz à effet de serre évitées',
 'Nombre de bonus octroyés à des véhicules électriques et hybrides rechargeables' : 'Nombre de bonus octroyés à des véhicules électriques',
 "Quantité de matières plastiques évitées ou dont le recyclage ou l'intégration a été soutenue" : 'Quantité de matières plastiques évitées',
 #'Montant total des travaux associés aux dossiers validés' : 'Montant total des travaux',
 'Nombre de nouveaux projets (nouvelle ligne, extension de ligne et pôle)' : 'Nombre de nouveaux projets',
 'Montant de l’investissement total déclenché' : 'Montant de l’investissement total',
 'Nombre de projets de tourisme durable financés' : 'Nombre de projets de tourisme durable financés',
 'Nombre de projets de rénovation de cathédrales et de monuments nationaux initiés' : 'Nombre de projets de rénovation',
 'Montant total investi pour la rénovation de monuments historiques appartenant aux collectivités territoriales' :'Montant total investi pour la rénovation',
 'Nombre de projets de rénovation de monuments historiques appartenant aux collectivités territoriales bénéficiaires initiés' : 'Nombre de projets de rénovation',
 "Nombre de contrats de professionnalisation bénéficiaires de l'aide exceptionnelle" : 'Nombre de contrats de professionnalisation',
 "Nombre de contrats d'apprentissage bénéficiaires de l'aide exceptionnelle" : 'Nombre de contrats d’apprentissage',
 "Nombre d'aides à l'embauche des travailleurs handicapés": "Nombre d'aides à l'embauche des travailleurs handicapés",
 'Nombre de projets locaux soutenus  (rénovation, extension, création de lignes' :'Nombre de projets locaux soutenus',
 #'Nombre de dossiers MaPrimeRénov validés': 'Nombre de dossiers MaPrimeRénov bénéficiaires ("particulier" et "copropriété")',
 #'Nombre de dossiers MaPrimeRénov bénéficiaires ("particulier" et "copropriété")': 'Nombre de bénéficiaires',
 "Nombre de dossiers MaPrimeRénov' payés": "Nombre de dossiers payés",
  "Nombre d'entreprises bénéficiares": "Nombre d'entreprises bénéficiaires"
                  }

In [187]:
df_dep.short_indic = df_dep.short_indic.apply(lambda x: dict_indicateur[x] if x in dict_indicateur else x)
df_dep.short_indic = df_dep.short_indic.apply(lambda x: dict_indicateur[x] if x in dict_indicateur else x)
df_dep.short_indic = df_dep.short_indic.apply(lambda x: dict_indicateur[x] if x in dict_indicateur else x)

df_reg.short_indic = df_reg.short_indic.apply(lambda x: dict_indicateur[x] if x in dict_indicateur else x)
df_reg.short_indic = df_reg.short_indic.apply(lambda x: dict_indicateur[x] if x in dict_indicateur else x)
df_reg.short_indic = df_reg.short_indic.apply(lambda x: dict_indicateur[x] if x in dict_indicateur else x)

df_nat2.short_indic =df_nat2.short_indic.apply(lambda x: dict_indicateur[x] if x in dict_indicateur else x)
df_nat2.short_indic =df_nat2.short_indic.apply(lambda x: dict_indicateur[x] if x in dict_indicateur else x)
df_nat2.short_indic =df_nat2.short_indic.apply(lambda x: dict_indicateur[x] if x in dict_indicateur else x)

In [188]:
df.rename(columns={"effect_id":"indicateur"}, inplace=True)
df.indicateur = df.indicateur.str.strip()

In [189]:
dict_mesures  = {
 "Appels à projets dédiés à l'efficacité énergétique et à l'évolution des procédés en faveur de la décarbonation de l'industrie": "AAP Efficacité énergétique",
 "CIE jeunes" : "Contrats Initiatives Emploi (CIE) Jeunes",
 'France Num' : 'France Num : aide à la numérisation des TPE,PME,ETI',
 'Guichet efficacité énergétique dans industrie' : 'Guichet efficacité énergétique',
 "Modernisation des filières automobiles et aéronautiques" : "Modernisation des filières automobiles et aéronautiques",
 "PEC jeunes": "Parcours emploi compétences (PEC) Jeunes",
 'Relocalisation : soutien aux projets industriels dans les territoires' : 'AAP Industrie : Soutien aux projets industriels territoires',
 'Relocalisation : sécurisation des approvisionnements critiques' : 'AAP Industrie : Sécurisation approvisionnements critiques',
 'Renforcement des subventions de Business France (chèque export, chèque VIE)' : 'Renforcement subventions Business France',
 "Soutien à la modernisation industrielle et renforcement des compétences dans la filière nucléaire" : "AAP industrie : modernisation industrielle et renforcement des compétences dans la filière nucléaire",
 'Soutien à la recherche aéronautique civil' :'Soutien recherche aéronautique civil',
 'Rénovation bâtiments Etats' : 'Rénovation des bâtiments Etats (marchés notifiés)'
}

In [190]:
df_dep["short_mesure"] = df_dep.mesure.apply(lambda x: dict_mesures[x] if x in dict_mesures else x)
df_reg["short_mesure"] = df_dep.mesure.apply(lambda x: dict_mesures[x] if x in dict_mesures else x)
df_nat2["short_mesure"] = df_dep.mesure.apply(lambda x: dict_mesures[x] if x in dict_mesures else x)

In [191]:
def get_df_sum_indicator(df_dep, indicators_to_sum, new_indicator, new_indic, new_mesure):
    df_dep = df_dep.copy()
    df_temp = df_dep.loc[df_dep.indicateur.str.contains(indicators_to_sum, regex=True)].groupby(["Date", "dep"]).sum().copy()
    df_temp["indicateur"] = new_indicator + " - " + new_indic
    df_temp["short_indic"] = new_indicator
    df_temp["mesure"] = new_mesure
    df_temp = (df_temp
            .merge(df_dep.drop(columns=["indicateur", "indic_id", "short_indic", "mesure", "short_mesure", "valeur"])
                   .drop_duplicates(["Date", "dep"]), 
             on=["Date", "dep"]))
    df_temp["short_mesure"] = new_mesure
    df_temp["indic_id"] = new_indic
    #df_temp.fillna("NaN", inplace=True)
    return df_temp

In [192]:
df_temp = get_df_sum_indicator(
    df_dep, 
    "PEE3|EEI1|CBC3",
    "Nombre d’entreprises ayant reçu l’aide",
    "SSS1",
    "Décarbonation de l'industrie (Appel à projets EE + Guichet EE + Chaleur bas carbone)")
df_temp2 = get_df_sum_indicator(
    df_dep, 
    "PEE2|CBC2",
    "Montant cumulé de l’investissement total ainsi déclenché",
    "SSS2",
    "Décarbonation de l'industrie (Appel à projets EE + Guichet EE + Chaleur bas carbone)")

In [193]:
df_dep = pd.concat([df_dep, df_temp, df_temp2])

In [194]:
def check_dep_coherence(df):
    assert df['dep'].isnull().sum() == 0, "Certaines lignes ne possèdent pas de code département."
    
    deps_test = sorted(df['dep'].unique())
    deps_true = sorted(taxo_dep_df['dep'].unique())
    assert deps_test == deps_true, "Départements ne concordent pas."
    
    assert sorted(df['libelle'].unique()) == sorted(taxo_dep_df['libelle'])
    

def check_reg_coherence(df):
    assert df['reg'].isnull().sum() == 0, "Certaines lignes ne possèdent pas de code région."
    
    regs_test = sorted(df['reg'].unique())
    regs_true = sorted(taxo_reg_df['reg'].unique())
    assert regs_test == regs_true, "Régions ne concordent pas."
        
    
def check_df_dep_enr_format(df_dep_enr):
    assert df_dep_enr.isnull().sum().sum() == 0
    assert df_dep_enr.shape[0] == df_dep.shape[0]

In [195]:
check_dep_coherence(df_dep)
check_reg_coherence(df_dep)
check_df_dep_enr_format(df_dep)

In [196]:
df_dep.to_csv("pp_dep.csv", sep=";")

In [197]:
df_nat.to_csv("pp_nat.csv", sep=";")

In [198]:
df_nat2["maille"] = "Nationale"
df_reg["maille"] = "Régionale"
df_reg = df_reg.merge(taxo_reg_df[["nccenr", "reg"]], how='left', left_on='departement', right_on='nccenr')
df_reg = df_reg.drop(["nccenr", 'reg_x'], axis=1)
df_dep["maille"] = "Départementale"

df_dep = df_dep.rename(columns={"departement": "localisation", 'dep': 'Code_Departement', 'reg': 'Code_Region'})
df_reg = df_reg.rename(columns={"departement": "localisation", 'reg_y': 'Code_Region'})
df_nat2 = df_nat2.rename(columns={"mesure": "localisation", "departement": "mesure"})
df_nat2["localisation"] = "Nationale"

In [199]:
df_all = pd.concat([df_dep, df_reg, df_nat2])
df_all.reset_index(drop=True, inplace=True)

In [200]:
columns = ['dep', 'reg', 'libelle', 'region', 'state_id', 'short_mesure']
df_all.drop(columns, inplace=True, axis=1)
df_all = df_all.rename(columns={'period_month_tri': 'abrev_mois'})

In [201]:
def recup_date(string: str) -> str:
    return string[:10]

df_all["period_date"] = df_all["period_date"].apply(recup_date)


In [202]:
clef_indicateur2volet = {
    "ecologie": ["RBC1",
                 "RBC2",
                 "RBC3",
                 "RBC4",
                 "IFO1",
                 "CBC1",
                 "CBC2",
                 "CBC3",
                 "AEA1",
                 "AEA2",
                 "CIC1",
                 "MDA1",
                 "FFR1",
                 "REA1",
                 "FRE1",
                 "BEL1",
                 "BEL2",
                 "DVP1",
                 "PSP1",
                 "MPR1",
                 "MPR2",
                 "MPR3",
                 "MPR4",
                 "MPR5",
                 "PEE1",
                 "PEE2",
                 "PEE3",
                 "MCT1",
                 "MCT2",
                 "MCT3",
                 "VNA1",
                 "VNA2",
                 "AMD1",
                 "TSA1",
                 "TDN1",
                 "RER1",
                 "DGA1",
                 "VDP1",
                 "VDP2",
                 "FRE1",
                 "FRE2",
                 "PVG1",
                 "PVG2",
                 "IEP1",
                 "TCI1",
                 "EOM1",
                 "CIR1",
                 "CIR2",
                 "DVD1",
                 "DVD2",
                 "DVD3",
                 "DVD4",
                 "MIR1",
                 "MIR2",
                 "SPL1",
                 "MRB1",
                 "IRR1",
                 "RRE1",
                 "RRE2",
                 "BPI1",
                 "BPI2",
                 "BPI3",
                 "BPI4",
                 "ETE1",
                 "ETE2",
                 "MTV1",
                 "MTV2",
                 "MTV3",
                 "MTV4",
                 "EEI1",
                 "EEI2",
                 "FAA1",
                 "FAA2",
                 "FAA3",
                 "PAM1",
                 "VAD1",
                 "VAI1",
                 "MDS1",
                 "PPR1",
                 "RAS1",
                 "RAS2",
                 "RBE1",
                 "BOE1",
                 "ETH1",
                 "ETH2",
                 "FVE1",
                 "FVE2",
                 "DRA1",
                 "DRA2",
                 "DRA3",
                 "DRA4",
                 "DRA5",
                 "DRA6",
                 "DRA7",
                 "DRA8"],
    "competitivite": ["ICC1",
                      "ICC2",
                      "ICC3",
                      "ICC4",
                      "IFP1",
                      "IFP2",
                      "IFP3",
                      "IFP4",
                      "IFS1",
                      "IFS2",
                      "IFS3",
                      "IFS4",
                      "MNS1",
                      "MNS2",
                      "MNS3",
                      "MNS4",
                      "GFP1",
                      "GFP2",
                      "GFP3",
                      "GFP4",
                      "AFG1",
                      "AFG2",
                      "AFG3",
                      "SFC1",
                      "PPA1",
                      "PPA2",
                      "IDF1",
                      "IDF2",
                      "IDF3",
                      "BIP1",
                      "BIP2",
                      "ROF1",
                      "ROF2",
                      "APS1",
                      "APS2",
                      "APS3",
                      "PIT1",
                      "PIT2",
                      "PIT3",
                      "SAC1",
                      "SAC2",
                      "SAC3",
                      "PAT1",
                      "PAT2",
                      "FUM1",
                      "APR1",
                      "SBF1",
                      "SBF2",
                      "ERD1",
                      "SNE1",
                      "PAE1",
                      "SSV1",
                      "PAT1"],
    "cohesion": ["CIE1",
                 "CIE2",
                 "PEC1",
                 "GJE1",
                 "PQU1",
                 "RRU1",
                 "APL1",
                 "RDC1",
                 "PIA1",
                 "PIA2",
                 "PIA3",
                 "PIA4",
                 "PEJ1",
                 "PTH1",
                 "SCI1",
                 "OFO1",
                 "CPE1",
                 "PRR1",
                 "PRR2",
                 "INU1",
                 "APP1",
                 "SNU1",
                 "DFO1",
                 "CDP1",
                 "PRN1",
                 "THD1",
                 "VEF1",
                 "SMS1",
                 "IEX1",
                 "TDU1",
                 "SIL1",
                 "SHO1",
                 "SHO2",
                 "PVU1",
                 "FNE1",
                 "PJP1",
                 "IAE1",
                 "CDR1"]
}


In [203]:
ecologie2indic = clef_indicateur2volet["ecologie"]
competitivite2indic = clef_indicateur2volet["competitivite"]
cohesion2indic = clef_indicateur2volet["cohesion"]
df1 = pd.DataFrame(ecologie2indic, columns=["code"])
df1["volet"] = "Ecologie"

df2 = pd.DataFrame(competitivite2indic, columns=["code"])
df2["volet"] = "Compétitivité"

df3 = pd.DataFrame(cohesion2indic, columns=["code"])
df3["volet"] = "Cohésion"

df = pd.concat([df1, df2, df3])

In [204]:
df_all = df_all.merge(df, how="left", left_on="indic_id", right_on="code")

In [205]:
df_mesure = pd.read_csv("refs/INDICATEURS.csv", sep=';')
L = [["DVP1", "Transition écologique, transports, et logement"], ["ETE1","Transition écologique, transports, et logement"], ["ETE2","Transition écologique, transports, et logement"]]
for mesure in df.code.unique():
    code = mesure[:-1]
    if code in df_mesure["Code-4"].unique():
        L += [[mesure, df_mesure[df_mesure["Code-4"] == code]["Ministères"].iloc[0]]]
    
df_to_merge = pd.DataFrame(L, columns=["code", "Ministères"])
df_all = df_all.merge(df_to_merge, how='left', left_on='code', right_on='code')

In [206]:
df_all

,indicateur,period_date,abrev_mois,Date,valeur,short_indic,indic_id,localisation,mesure,Code_Departement,Code_Region,maille,code,volet,Ministères
0,Nombre de locaux commerciaux connectables à la...,2020-12-31,Dec,Décembre 2020,190200.0,Nombre de locaux commerciaux connectables à la...,THD1,Sarthe,Généralisation de la fibre optique,72,52,Départementale,THD1,Cohésion,Cohésion des territoires et ville
1,Nombre de locaux commerciaux connectables à la...,2020-12-31,Dec,Décembre 2020,31000.0,Nombre de locaux commerciaux connectables à la...,THD1,Ardèche,Généralisation de la fibre optique,07,84,Départementale,THD1,Cohésion,Cohésion des territoires et ville
2,Nombre de locaux commerciaux connectables à la...,2020-12-31,Dec,Décembre 2020,26000.0,Nombre de locaux commerciaux connectables à la...,THD1,Lozère,Généralisation de la fibre optique,48,76,Départementale,THD1,Cohésion,Cohésion des territoires et ville
3,Nombre de locaux commerciaux connectables à la...,2020-12-31,Dec,Décembre 2020,176000.0,Nombre de locaux commerciaux connectables à la...,THD1,Pyrénées-Orientales,Généralisation de la fibre optique,66,76,Départementale,THD1,Cohésion,Cohésion des territoires et ville
4,Nombre de locaux commerciaux connectables à la...,2020-12-31,Dec,Décembre 2020,54100.0,Nombre de locaux commerciaux connectables à la...,THD1,Corse-du-Sud,Généralisation de la fibre optique,2A,94,Départementale,THD1,Cohésion,Cohésion des territoires et ville
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58954,Taux d'agents équipés pour le travail à distan...,2021-04-30,Apr,Avril 2021,68.0,Taux d'agents équipés pour le travail à distance,PTA1,Nationale,Nouveau poste de travail de l’agent public,NaN,NaN,Nationale,NaN,NaN,NaN
58955,Taux d'agents équipés pour le travail à distan...,2021-05-31,May,Mai 2021,68.0,Taux d'agents équipés pour le travail à distance,PTA1,Nationale,Nouveau poste de travail de l’agent public,NaN,NaN,Nationale,NaN,NaN,NaN
58956,Taux d'agents équipés pour le travail à distan...,2021-06-30,Jun,Juin 2021,68.0,Taux d'agents équipés pour le travail à distance,PTA1,Nationale,Nouveau poste de travail de l’agent public,NaN,NaN,Nationale,NaN,NaN,NaN
58957,Taux d'agents équipés pour le travail à distan...,2021-07-31,Jul,Juillet 2021,68.0,Taux d'agents équipés pour le travail à distance,PTA1,Nationale,Nouveau poste de travail de l’agent public,NaN,NaN,Nationale,NaN,NaN,NaN


In [207]:
def mkdir_ifnotexist(path) :
    if not os.path.isdir(path) :
        os.mkdir(path)
mkdir_ifnotexist('exports')
df_all.to_csv(os.path.join('exports', 'propilot.csv'), sep=';')

In [211]:
df_all[df_all["Ministères"]=="Transition écologique, transports, et logement"].mesure.unique()

array(['Ecoconception et entreprises engagées pour la transition écologique : EETE',
       'Soutien à chaleur bas carbone',
       'Réhabilitation Friches (urbaines et sites pollués)',
       'Hydrogène – Ecosystèmes territoriaux',
       'Investissement dans le recyclage et le réemploi (dont accompagnement filière plastique)',
       'Rénovation énergétique et réhabilitation lourde des logements sociaux',
       'Restauration écologique',
       "Guichet efficacité énergétique dans l'industrie",
       "Appels à projets dédiés à l'efficacité énergétique et à l'évolution des procédés en faveur de la décarbonation de l'industrie",
       'recyclage et valorisation des déchets', 'Tourisme durable',
       'Bonus écologique', 'Soutien à la recherche aéronautique civil',
       'Prime à la conversion des véhicules légers', "MaPrimeRénov'",
       'Transports en commun Ile-de-France', 'Soutien petites lignes',
       'Frêt',
       'Aide à la relance de la construction durable (maires dens